<a href="https://colab.research.google.com/github/bandreibal/03MIAR---Algoritmos-de-Optimizacion---2023/blob/main/SEMINARIO/Seminario_Algoritmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario
*Nombre y Apellidos*: Bogdan Andrei Baltes

*Github*: https://github.com/bandreibal/03MIAR---Algoritmos-de-Optimizacion---2023/tree/main/SEMINARIO

*Colab*:   https://colab.research.google.com/drive/1tx2Jd3odbEdi0YGZtU3-k25M-cJJ3Qh8?usp=sharing

*Problema*:
**Organizar los horarios de partidos de La Liga**

*Descripción del problema*:
Desde la La Liga de fútbol profesional se pretende organizar los horarios de los partidos de liga de cada jornada. Se conocen algunos datos que nos deben llevar a diseñar un algoritmo que realice la asignación de los partidos a los horarios de forma que **maximice la audiencia**.





                                        

# (*) ¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>

Estamos ante una **variación con repetición**, por lo que tendremos $n^k$ posibles soluciones. En este caso, $n$ representa los posibles horarios para su celebración, y $k$ es el número de partidos (o parejas de equipos que se enfrentan). Esto se debe a que el orden de los partidos importa, y puede haber varios partidos en el mismo horario.

Por tanto, tendremos $10^{10} = 10.000.000.000$ posibilidades de soluciones sin tener en cuenta las restricciones.

# ¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones?


En este caso, la única restricción es aquella de tener obligatoriamente al menos un partido en el horario del viernes a las 20h y del lunes a las 20h.

Por lo tanto, para cumplir con la restricción, reservaremos dos partidos cualesquiera para los dos horarios.

Las posibles combinaciones para esta reserva será una **variación sin repetición**, que dará lugar a $\frac{n!}{(n-k)!}$ posibilidades. En este caso, la $n$ corresponderá al número total de partidos: 10. La $k$ será igual a 2, el número de horarios entre los que queremos repartir dos de los 10 partidos. Por tanto:
$\frac{n!}{(n-k)!} = 90$.

En cuanto a los otros ocho partidos, estaremos ante la misma situación que en el caso sin restricciones, pero con una $n$ menor. Será una **variación con repetición**, dando lugar a $10^{8} = 100.000.000$ posibles soluciones.

Por tanto, multiplicando estos dos resultados, se tendrían $90 \cdot 10^{8} = 9.000.000.000$ posibilidades.

# (*) ¿Cual es la estructura de datos que mejor se adapta al problema? Arguméntalo.
*(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguméntalo.)*


Para presentar los resultados de forma ordenada, se optó por crear una clase llamada *Match* con los atributos *slot*, *team1* y *team2*, para indicar el horario del partido y los equipos que compiten en él. Por tanto, **todos los algoritmos devuelven una lista de objetos *Match***.

Sin embargo, estos objetos solo se usaron en todos los algoritmos en un intento de estandarización de los resultados. En el algoritmo genético, se usó desde el principio el objeto *Match*. En los otros dos, las estructuras de datos empleadas para llegar a ellos fueron o bien listas, o combinaciones de listas con otros elementos (por ejemplo, en diccionarios).
- En el algoritmo por fuerza bruta, se emplea una **lista de listas**, que empieza estando vacía: [[], [], [], [], [], [], [], [], [], []]. Cada una de las listas interiores representa uno de los diez posibles horarios. Cuando se producen las asignaciones, se añaden los elementos a la lista correspondiente al horario deseado. Se mantiene siempre el mismo orden, siendo así la primera lista el horario de viernes a las 20h, y la última lunes a las 20h.
- En el algoritmo voraz, se emplea un **diccionario que contiene listas**. Un ejemplo de elemento de este diccionario es: 'V20': [0.4, []]. La clave será el horario del partido, y el valor es una lista con dos elementos: el coeficiente relacionado con el horario, y las parejas de equipos que jugarán en ese horario.
- **En el algoritmo genético, se empleó la clase *Match*** para facilitar las acciones de los operadores genéticos, pudiendo modificar los horarios de forma más sencilla en los cruces y las mutaciones.

Al principio, al modelar el problema, también se había considerado usar un formato tabular para los datos. Sin embargo, se encontraron dificultades a la hora de representar aspectos como las coincidencias horarias. Esto fue más sencillo mediante las listas, pudiendo así acceder a esta información consultando la longitud de la lista.

Sería relativamente sencillo modificar las estructuras de las listas de los dos primeros algoritmos, pero se optó por estandarizar solamente la salida para mostrar el proceso de diseño llevado a cabo durante la práctica.

# (*) ¿Cuál es la función objetivo?

La función objetivo es: $$Z = \sum_{i \in SLOTS}^{len(SLOTS)} {Base\_Cat} \cdot {Coef\_Slot} \cdot {Coef\_Coinc}$$

donde:

- $Base\_Cat$ indicará la suma de la audiencia base de los partidos colocados en el horario, en base a la categoría de los integrantes de cada pareja.
- $Coef\_Slot$ es el coeficiente por el que se debe multiplicar la audiencia estadística en función del día y hora del horario.
- $Coef\_Coinc$ es el valor por el que se multiplicará como penalización por coincidencia de horarios, en función del número de partidos que ocurren en el mismo *slot* de tiempo.

# (*) ¿Es un problema de maximización o minimización?

Se trata de un **problema de maximización**. El objetivo final del problema que consiste en asignar los partidos a los diferentes horarios es maximizar la audiencia de dichos partidos.

# Diseña un algoritmo para resolver el problema por fuerza bruta.

*Nota*: Se han dejado algunos prints en forma de comentarios en caso de querer ver el funcionamiento interno del algoritmo. Si es el caso, bastaría con descomentar esos comentarios y ejecutar de nuevo.

In [ ]:
import itertools
import copy

class Match:
    def __init__(self, team1, team2, slot):
        self.team1 = team1
        self.team2 = team2
        self.slot = slot

def planificador_bruta(MATCHES):

    global_best_combination = []
    global_best_audience = 0
    count = 0

    SLOTS_COEFFICIENTS = {
        'V20': 0.4,
        'S12': 0.55,
        'S16': 0.7,
        'S18': 0.8,
        'S20': 1.0,
        'D12': 0.45,
        'D16': 0.75,
        'D18': 0.85,
        'D20': 1.0,
        'L20': 0.4
    }

    SLOTS_COEFFICIENTS_LIST = list(SLOTS_COEFFICIENTS.values())

    BASE_AUDIENCE = {
        ('A', 'A'): 2.0,
        ('A', 'B'): 1.3,
        ('A', 'C'): 1.0,
        ('B', 'B'): 0.9,
        ('B', 'C'): 0.75,
        ('C', 'C'): 0.47
    }

    PENALTY_COEFFICIENTS = {
        -1: 0.0,
        0: 1.0,
        1: 0.75,
        2: 0.55,
        3: 0.4,
        4: 0.3,
        5: 0.25,
        6: 0.22,
        7: 0.20,
        8: 0.20
    }

    EQUIPOS_CAT = {
        'Real Madrid': 'A',
        'R. Sociedad': 'A',
        'Barcelona': 'A',
        'Celta': 'B',
        'Valencia': 'B',
        'Athletic': 'B',
        'Villarreal': 'B',
        'Alavés': 'B',
        'Levante': 'B',
        'Espanyol': 'B',
        'Sevilla': 'B',
        'Betis': 'B',
        'Atlético': 'B',
        'Getafe': 'B',
        'Mallorca': 'C',
        'Eibar': 'C',
        'Leganés': 'C',
        'Osasuna': 'C',
        'Granada': 'C',
        'Valladolid': 'C',
    }

    # Los nombres de los equipos se traducen a sus categorías, ordenadas dentro
    # de la misma tupla para poder utilizar la audiencia de BASE_AUDIENCE
    result = [(EQUIPOS_CAT[team1], EQUIPOS_CAT[team2]) for team1, team2 in MATCHES]
    sorted_matches = [tuple(sorted(match)) for match in result]

    # Guardamos la equivalencia de las parejas de equipos originales
    # a sus "traducciones"
    MATCHES_EQ = list(zip(MATCHES, sorted_matches))
    MATCHES_AUDIENCES = [BASE_AUDIENCE[x] for x in sorted_matches]

    # Se asignan las parejas de equipos a diferentes slots de manera recursiva,
    # guardando la mejor audiencia y la planificación que lleva a ella
    def assign_items(item_index, my_items):
        nonlocal best_audience
        nonlocal best_combination
        if item_index == len(my_items):
            current_audience = sum(sum(slot) * SLOTS_COEFFICIENTS_LIST[index] * PENALTY_COEFFICIENTS[(len(slot)-1)] for index, slot in enumerate(my_list))
            if current_audience > best_audience:
                best_audience = current_audience
                best_combination = copy.deepcopy(my_list)
            return

        for list_index in range(len(my_list)):
            if my_items[item_index] not in my_list[list_index]:
                my_list[list_index].append(my_items[item_index])
                assign_items(item_index + 1, NEW_MATCHES)
                my_list[list_index].remove(my_items[item_index])

        if len(my_list) < 10:
            my_list.append([my_items[item_index]])
            assign_items(item_index + 1, NEW_MATCHES)
            my_list.pop()

    # Se itera en cada combinación de opciones para partidos
    # en viernes y lunes (la restricción del problema)
    for v in itertools.permutations(MATCHES_AUDIENCES, 2):
        count += 1
        # print("Iteración #", count)
        NEW_MATCHES = copy.deepcopy(MATCHES_AUDIENCES)

        for element in list(v):
            if element in NEW_MATCHES:
                NEW_MATCHES.remove(element)

        # print("Partidos en V20 y L20:", v[0], v[1])
        # print("Partidos a repartir:", NEW_MATCHES)

        my_list = [[], [], [], [], [], [], [], [], [], []]
        my_list[0].append(v[0])
        my_list[9].append(v[1])
        # print(my_list)
        best_combination = []
        best_audience = 0

        assign_items(0, NEW_MATCHES)
        # print("Mejor audiencia: ", best_audience)
        # print("Mejor combinación: ", best_combination)

        if best_audience > global_best_audience:
                global_best_audience = best_audience
                global_best_combination = copy.deepcopy(best_combination)
        # print("-------------------------------------")

    # print("RESULTADOS GLOBALES:")
    # print("Mejor audiencia: ", global_best_audience)
    # print("Mejor combinación: ", global_best_combination)

    # Se estandarizan las salidas para guardar los resultados en objetos
    # Match y se prepara la presentación de los resultados finales
    for i in range(len(global_best_combination)):
        for j in range(len(global_best_combination[i])):
            value = global_best_combination[i][j]
            key = next(key for key, val in BASE_AUDIENCE.items() if val == value)
            global_best_combination[i][j] = key

    # print(global_best_combination)

    count = 0
    matches = []
    for slot in global_best_combination:
        for match in slot:
            slot_name = list(SLOTS_COEFFICIENTS.keys())[count]
            new_match = Match(match[0], match[1], slot_name)
            matches.append(new_match)
        count += 1

    print("Planificación mediante ALGORITMO POR FUERZA BRUTA\n-----")

    already_used_teams = []

    for m in matches:
        for names, eq in MATCHES_EQ:
            if (m.team1, m.team2) == eq and names not in already_used_teams:
                m.team1, m.team2 = names
                already_used_teams.append(names)
        print(f"Horario: {m.slot}, Equipos: {m.team1} - {m.team2}")
    print('-----\nMejor audiencia: ', global_best_audience, 'millones de espectadores')

En la siguiente celda se ejecuta el algoritmo por fuerza bruta desarrollado con una entrada de ejemplo. Esta entrada solo contiene 16 de los 20 equipos. Esto se debe a que para entradas de 18 equipos (9 partidos) el algoritmo tarda alrededor de 16 minutos en ejecutar, mientras que ya con 20 equipos (10 partidos) no terminó de ejecutar en ninguno de los intentos realizados.

In [ ]:
partidos_ejemplo = [
    ('Celta', 'Real Madrid'),
    ('Valencia', 'R. Sociedad'),
    ('Mallorca', 'Eibar'),
    ('Athletic', 'Barcelona'),
    ('Leganés', 'Osasuna'),
    ('Villarreal', 'Granada'),
    # ('Alavés', 'Levante'),
    # ('Espanyol', 'Sevilla'),
    ('Betis', 'Valladolid'),
    ('Atlético', 'Getafe')
]

planificador_bruta(partidos_ejemplo)

Planificación mediante ALGORITMO POR FUERZA BRUTA
-----
Horario: V20, Equipos: Mallorca - Eibar
Horario: S16, Equipos: Villarreal - Granada
Horario: S18, Equipos: Atlético - Getafe
Horario: S20, Equipos: Celta - Real Madrid
Horario: D16, Equipos: Betis - Valladolid
Horario: D18, Equipos: Valencia - R. Sociedad
Horario: D20, Equipos: Athletic - Barcelona
Horario: L20, Equipos: Leganés - Osasuna
-----
Mejor audiencia:  5.888499999999999 millones de espectadores


# Calcula la complejidad del algoritmo por fuerza bruta.

A alto nivel, este algoritmo está compuesto por dos bucles:
- El bucle exterior, como se mencionó anteriormente, es una variación sin repetición, que se ejecutará $\frac{n!}{(n-k)!}$ veces. Su objetivo es cumplir con el requerimiento de siempre tener un partido los viernes y los lunes a las 20:00. Por tanto, la $k$ será igual a 2, quedándonos con una expresión igual a $\frac{n!}{(n-k)!} = n \cdot (n-1)$. Esto nos dará una complejidad del bucle de $O(n^2)$.
- El bucle interior, a su vez, será una variación con repetición, y será el encargado de explorar las distintas formas de agendar los $n-2$ partidos restantes entre 10 posibles horarios. Por tanto, se recorrerán $10^{n-2}$ posibilidades, dejando el bucle con una complejidad de $O(10^{n})$.

La complejidad final del algoritmo será $O(10^{n})$, ya que es el término dominante, y se trata de una complejidad exponencial (comúnmente denotada por $O(2^{n})$). Es una de las complejidades más costosas en tiempo, solo peor que la factorial, y se debe tener en cuenta que habrá un número de posibles soluciones que es difícil de emplear por fuerza bruta.

# (*) Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta.
*Argumenta por qué crees que mejora el algoritmo por fuerza bruta.*

Los algoritmos por fuerza bruta tienen la evidente ventaja de que garantizan obtener siempre el óptimo global, ya que exploran todo el espacio de soluciones y comparan los resultados de todas ellas.

Sin embargo, los algoritmos por fuerza bruta suelen ser ineficientes, sobre todo llegando a números de elementos muy elevados. Por ejemplo, el algoritmo desarrollado en las secciones anteriores no llega a ejecutarse con $n = 10$. Para ahorrar memoria y tiempo, se han desarrollado otros dos algoritmos que no son exactos: un algoritmo voraz y un algoritmo genético. Son algoritmos que no garantizan un óptimo global pero que se aproximan bastante bien a los resultados del algoritmo por fuerza bruta, en un tiempo de ejecución mucho menor.

## Algoritmo Voraz
El algoritmo ordena todos los partidos en función de la audiencia base que pueden tener, de mayor a menor. Se trata de un algoritmo voraz porque siempre escoge las soluciones óptimas en cada paso.

Se itera por todos los partidos, calculando la audiencia que aportaría su planificación en cada uno de los horarios, teniendo en cuenta la audiencia base de las categorías de los equipos, la audiencia por el horario y la penalización por coincidencias con otros partidos. Tras estos cálculos, se elige el horario que mayor audiencia aporte.

In [ ]:
import random
import copy

class Match:
    def __init__(self, team1, team2, slot):
        self.team1 = team1
        self.team2 = team2
        self.slot = slot

def planificador_voraz(MATCHES):

    SLOTS_COEFFICIENTS = {
        'V20': [0.4, []],
        'S12': [0.55, []],
        'S16': [0.7, []],
        'S18': [0.8, []],
        'S20': [1.0, []],
        'D12': [0.45, []],
        'D16': [0.75, []],
        'D18': [0.85, []],
        'D20': [1.0, []],
        'L20': [0.4, []]
    }

    EQUIPOS_CAT = {
        'Real Madrid': 'A',
        'R. Sociedad': 'A',
        'Barcelona': 'A',
        'Celta': 'B',
        'Valencia': 'B',
        'Athletic': 'B',
        'Villarreal': 'B',
        'Alavés': 'B',
        'Levante': 'B',
        'Espanyol': 'B',
        'Sevilla': 'B',
        'Betis': 'B',
        'Atlético': 'B',
        'Getafe': 'B',
        'Mallorca': 'C',
        'Eibar': 'C',
        'Leganés': 'C',
        'Osasuna': 'C',
        'Granada': 'C',
        'Valladolid': 'C',
    }

    BASE_AUDIENCE = {
        ('A', 'A'): 2.0,
        ('A', 'B'): 1.3,
        ('A', 'C'): 1.0,
        ('B', 'B'): 0.9,
        ('B', 'C'): 0.75,
        ('C', 'C'): 0.47
    }

    PENALTY_COEFFICIENTS = {
        0: 1.0,
        1: 0.75,
        2: 0.55,
        3: 0.4,
        4: 0.3,
        5: 0.25,
        6: 0.22,
        7: 0.20,
        8: 0.20
    }

    # Los nombres de los equipos se traducen a sus categorías, ordenadas dentro
    # de la misma tupla para poder utilizar la audiencia de BASE_AUDIENCE
    result = [(EQUIPOS_CAT[team1], EQUIPOS_CAT[team2]) for team1, team2 in MATCHES]
    sorted_matches = [tuple(sorted(match)) for match in result]

    # Guardamos la equivalencia de las parejas de equipos originales
    # a sus "traducciones"
    MATCHES_EQ = list(zip(MATCHES, sorted_matches))

    # Se ordenan los partidos de mayor a menor audiencia
    ordered_matches = sorted(sorted_matches, key=lambda x: BASE_AUDIENCE.get(x, 0), reverse=True)

    total_audience, new_best_audience, count = 0, 0, len(ordered_matches)

    for match in ordered_matches:
        slot_audiences = []

        # Se comprueba primero si no se cumple la restricción de tener al
        # menos un partido en horario V20 y L20
        if count == 2 and len(SLOTS_COEFFICIENTS['V20'][1]) == 0 and len(SLOTS_COEFFICIENTS['L20'][1]) == 0:
            new_best_audience = SLOTS_COEFFICIENTS['V20'][0] * BASE_AUDIENCE[match]
            NEW_BEST_CONFIG = copy.deepcopy(SLOTS_COEFFICIENTS)
            NEW_BEST_CONFIG['V20'][1].append(BASE_AUDIENCE[match])

        elif count == 1 and len(SLOTS_COEFFICIENTS['V20'][1]) == 0:
            new_best_audience = SLOTS_COEFFICIENTS['V20'][0] * BASE_AUDIENCE[match]
            NEW_BEST_CONFIG = copy.deepcopy(SLOTS_COEFFICIENTS)
            NEW_BEST_CONFIG['V20'][1].append(BASE_AUDIENCE[match])

        elif count == 1 and len(SLOTS_COEFFICIENTS['L20'][1]) == 0:
            new_best_audience = SLOTS_COEFFICIENTS['L20'][0] * BASE_AUDIENCE[match]
            NEW_BEST_CONFIG = copy.deepcopy(SLOTS_COEFFICIENTS)
            NEW_BEST_CONFIG['L20'][1].append(BASE_AUDIENCE[match])

        else:
            # Se recorren todas las posibles asignaciones del partido a un horario
            for slot, data in SLOTS_COEFFICIENTS.items():
                new_audience = data[0] * BASE_AUDIENCE[match]
                if len(data[1]) > 0:
                    new_audience = new_audience * PENALTY_COEFFICIENTS[len(data[1])] - sum(data[1]) * (1 - PENALTY_COEFFICIENTS[len(data[1])])
                slot_audiences.append(new_audience)
                # Si la asignación es la que más audiencia generaría, es la nueva mejor solución
                if new_audience == max(slot_audiences):
                    new_best_audience = new_audience
                    NEW_BEST_CONFIG = copy.deepcopy(SLOTS_COEFFICIENTS)
                    NEW_BEST_CONFIG[slot][1].append(BASE_AUDIENCE[match])

        # Se actualiza la audiencia total y la planificación de los partidos
        SLOTS_COEFFICIENTS = copy.deepcopy(NEW_BEST_CONFIG)
        total_audience += new_best_audience
        count -= 1

    # Se estandarizan las salidas para guardar los resultados en objetos
    # Match y se prepara la presentación de los resultados finales
    configuraciones = copy.deepcopy(SLOTS_COEFFICIENTS)
    slots = []

    for slot, info in configuraciones.items():
        slots.append(info[1])

    for i in range(len(slots)):
        for j in range(len(slots[i])):
            value = slots[i][j]
            key = next(key for key, val in BASE_AUDIENCE.items() if val == value)
            slots[i][j] = key

    count = 0
    matches = []
    for slot in slots:
        for match in slot:
            slot_name = list(SLOTS_COEFFICIENTS.keys())[count]
            new_match = Match(match[0], match[1], slot_name)
            matches.append(new_match)
        count += 1

    print("Planificación mediante ALGORITMO VORAZ\n-----")

    already_used_teams = []

    for m in matches:
        for names, eq in MATCHES_EQ:
            if (m.team1, m.team2) == eq and names not in already_used_teams:
                m.team1, m.team2 = names
                already_used_teams.append(names)
        print(f"Horario: {m.slot}, Equipos: {m.team1} - {m.team2}")
    print('-----\nMejor audiencia: ', total_audience, 'millones de espectadores')

En la siguiente celda se ejecuta el algoritmo voraz desarrollado con una entrada de ejemplo. Esta entrada solo contiene 16 de los 20 equipos y es la misma que la que se usa en el algoritmo por fuerza bruta, para poder comparar los resultados.

In [ ]:
partidos_ejemplo = [
    ('Celta', 'Real Madrid'),
    ('Valencia', 'R. Sociedad'),
    ('Mallorca', 'Eibar'),
    ('Athletic', 'Barcelona'),
    ('Leganés', 'Osasuna'),
    ('Villarreal', 'Granada'),
    # ('Alavés', 'Levante'),
    # ('Espanyol', 'Sevilla'),
    ('Betis', 'Valladolid'),
    ('Atlético', 'Getafe')
]

planificador_voraz(partidos_ejemplo)

Planificación mediante ALGORITMO VORAZ
-----
Horario: V20, Equipos: Mallorca - Eibar
Horario: S16, Equipos: Villarreal - Granada
Horario: S18, Equipos: Atlético - Getafe
Horario: S20, Equipos: Celta - Real Madrid
Horario: D16, Equipos: Betis - Valladolid
Horario: D18, Equipos: Valencia - R. Sociedad
Horario: D20, Equipos: Athletic - Barcelona
Horario: L20, Equipos: Leganés - Osasuna
-----
Mejor audiencia:  5.888499999999999 millones de espectadores


En la siguiente celda se ejecuta con los 10 partidos de la documentación del problema.

In [ ]:
partidos_ejemplo = [
    ('Celta', 'Real Madrid'),
    ('Valencia', 'R. Sociedad'),
    ('Mallorca', 'Eibar'),
    ('Athletic', 'Barcelona'),
    ('Leganés', 'Osasuna'),
    ('Villarreal', 'Granada'),
    ('Alavés', 'Levante'),
    ('Espanyol', 'Sevilla'),
    ('Betis', 'Valladolid'),
    ('Atlético', 'Getafe')
]

planificador_voraz(partidos_ejemplo)

Planificación mediante ALGORITMO VORAZ
-----
Horario: V20, Equipos: Mallorca - Eibar
Horario: S12, Equipos: Villarreal - Granada
Horario: S16, Equipos: Alavés - Levante
Horario: S18, Equipos: Espanyol - Sevilla
Horario: S20, Equipos: Celta - Real Madrid
Horario: D12, Equipos: Betis - Valladolid
Horario: D16, Equipos: Atlético - Getafe
Horario: D18, Equipos: Valencia - R. Sociedad
Horario: D20, Equipos: Athletic - Barcelona
Horario: L20, Equipos: Leganés - Osasuna
-----
Mejor audiencia:  6.855999999999999 millones de espectadores


## Algoritmo Genético
El algoritmo genera una población inicial aleatoria que va cruzando y mutando a lo largo de las generaciones para encontrar la mejor solución.

In [ ]:
import random
import copy

class Match:
    def __init__(self, team1, team2, slot):
        self.team1 = team1
        self.team2 = team2
        self.slot = slot

def planificador_genetico(MATCHES):

    BASE_AUDIENCE = {
        ('A', 'A'): 2.0,
        ('A', 'B'): 1.3,
        ('A', 'C'): 1.0,
        ('B', 'B'): 0.9,
        ('B', 'C'): 0.75,
        ('C', 'C'): 0.47,
        ('', ''): 0.00
    }

    SLOTS_COEFFICIENTS = {
        'V20': 0.4,
        'S12': 0.55,
        'S16': 0.7,
        'S18': 0.8,
        'S20': 1.0,
        'D12': 0.45,
        'D16': 0.75,
        'D18': 0.85,
        'D20': 1.0,
        'L20': 0.4,
    }

    PENALTY_COEFFICIENTS = {
        0: 1.0,
        1: 0.75,
        2: 0.55,
        3: 0.4,
        4: 0.3,
        5: 0.25,
        6: 0.22,
        7: 0.20,
        8: 0.20
    }

    EQUIPOS_CAT = {
        'Real Madrid': 'A',
        'R. Sociedad': 'A',
        'Barcelona': 'A',
        'Celta': 'B',
        'Valencia': 'B',
        'Athletic': 'B',
        'Villarreal': 'B',
        'Alavés': 'B',
        'Levante': 'B',
        'Espanyol': 'B',
        'Sevilla': 'B',
        'Betis': 'B',
        'Atlético': 'B',
        'Getafe': 'B',
        'Mallorca': 'C',
        'Eibar': 'C',
        'Leganés': 'C',
        'Osasuna': 'C',
        'Granada': 'C',
        'Valladolid': 'C',
    }

    # Los nombres de los equipos se traducen a sus categorías, ordenadas dentro
    # de la misma tupla para poder utilizar la audiencia de BASE_AUDIENCE
    result = [(EQUIPOS_CAT[team1], EQUIPOS_CAT[team2]) for team1, team2 in MATCHES]
    sorted_matches = [tuple(sorted(match)) for match in result]

    # Guardamos la equivalencia de las parejas de equipos originales
    # a sus "traducciones"
    MATCHES_EQ = list(zip(MATCHES, sorted_matches))

    # Se ordenan los partidos de mayor a menor audiencia
    ordered_matches = sorted(sorted_matches, key=lambda x: BASE_AUDIENCE.get(x, 0), reverse=True)

    # Se fijan los parámetros del Algoritmo Genético
    POPULATION_SIZE = 100
    NUM_GENERATIONS = 1000
    TOURNAMENT_SIZE = 50
    MUTATION_RATE = 0.1

    # Se crea una función para comprobar que siempre haya (al menos) un partido
    # en horarios V20 y L20
    def required_slots(matches):
        has_v20_pair = any(match.slot == 'V20' for match in matches if match.team1 != '')

        if not has_v20_pair:
            non_v20_match = random.choice([match for match in matches if match.slot != 'V20'])
            non_v20_match.slot = 'V20'

        has_l20_pair = any(match.slot == 'L20' for match in matches if match.team1 != '')

        if not has_l20_pair:
            non_l20_match = random.choice([match for match in matches if match.slot != 'V20' and match.slot != 'L20'])
            non_l20_match.slot = 'L20'

        return matches

    # Se genera un cromosoma aleatorio, utilizando las parejas de equipos existentes y
    # asignando un horario aleatoriamente
    def generate_random_chromosome():
        matches = []
        available_matches = ordered_matches[:]
        for slot, _ in SLOTS_COEFFICIENTS.items():
            if available_matches:
                teams = random.choice(available_matches)
                available_matches.remove(teams)
                match = Match(teams[0], teams[1], slot)
                matches.append(match)

        # Se comprueba que haya al menos un partido en horarios V20 y L20
        matches = required_slots(matches)

        return matches

    # Se calcula la función fitness, iterando por todos los partidos dentro
    # de cada cromosoma para obtener la audiencia total
    def calculate_fitness(chromosome):
        total_audience = 0
        slot_coincidences = {}

        for match in chromosome:
            if match.slot in slot_coincidences:
                slot_coincidences[match.slot] += 1
            else:
                slot_coincidences[match.slot] = 0

        for match in chromosome:
            base_audience = BASE_AUDIENCE[(match.team1, match.team2)]
            slot_coefficient = SLOTS_COEFFICIENTS[match.slot]
            audience = base_audience * slot_coefficient * PENALTY_COEFFICIENTS[slot_coincidences[match.slot]]
            total_audience += audience

        return total_audience

    # Se seleccionan de una forma aleatoria dentro de la población a los que mayor
    # valor de audiencia tengan
    def tournament_selection(population):
        selected = []
        for _ in range(len(population)):
            participants = random.sample(population, TOURNAMENT_SIZE)
            selected.append(max(participants, key=lambda x: x['fitness']))
        return selected

    # El cruce es single-point, elegido al azar. Es decir, a partir de
    # una posición aleatoria, se intercambian los horarios de los padres
    def crossover(parent1, parent2):
        crossover_point = random.randint(1, len(parent1))
        child1 = copy.deepcopy(parent1)
        child2 = copy.deepcopy(parent2)

        for i in range(crossover_point):
            child1[i].slot = parent1[i].slot
            child2[i].slot = parent2[i].slot

        for i in range(crossover_point, len(parent1)):
            child1[i].slot = parent2[i].slot
            child2[i].slot = parent1[i].slot

        # Se comprueba que haya al menos un partido en horarios V20 y L20 en cada
        # individuo hijo
        child1 = required_slots(child1)
        child2 = required_slots(child2)

        return child1, child2

    # Se define la función de mutación. El individuo mutará si un número elegido
    # al azar supera el parámetro de MUTATION_RATE
    def mutate(chromosome):
        for i in range(len(chromosome)):
            if random.random() < MUTATION_RATE:
                chromosome[i].slot = random.choice(list(SLOTS_COEFFICIENTS.keys()))

        # Se comprueba que haya al menos un partido en horarios V20 y L20
        chromosome = required_slots(chromosome)

        return chromosome

    # Se inicializa la población
    population = []
    for _ in range(POPULATION_SIZE):
        chromosome = generate_random_chromosome()
        fitness = calculate_fitness(chromosome)
        population.append({'chromosome': chromosome, 'fitness': fitness})

    # Se crean las generaciones
    for generation in range(NUM_GENERATIONS):
        # Se seleccionan los individuos de la generación
        selected = tournament_selection(population)

        # Se hace el cruce
        offspring = []
        while len(offspring) < POPULATION_SIZE:
            parent1, parent2 = random.sample(selected, 2)
            child1, child2 = crossover(parent1['chromosome'], parent2['chromosome'])
            offspring1 = mutate(child1)
            offspring2 = mutate(child2)
            offspring.append({'chromosome': offspring1, 'fitness': calculate_fitness(offspring1)})
            offspring.append({'chromosome': offspring2, 'fitness': calculate_fitness(offspring2)})

        # Se actualiza la población y se mantienen solo los individuos con mayor valor de audiencia
        population = selected + offspring
        population = sorted(population, key=lambda x: x['fitness'], reverse=True)
        population = population[:POPULATION_SIZE]

    # Se selecciona la mejor solución
    best_solution = population[0]

    # Se prepara la salida de los resultados
    already_used_teams = []

    print("Planificación mediante ALGORITMO GENÉTICO\n-----")

    for m in best_solution['chromosome']:
        for names, eq in MATCHES_EQ:
            if (m.team1, m.team2) == eq and names not in already_used_teams:
                m.team1, m.team2 = names
                already_used_teams.append(names)
        print(f"Horario: {m.slot}, Equipos: {m.team1} - {m.team2}")
    print('-----\nMejor audiencia: ', best_solution['fitness'], 'millones de espectadores')

En la siguiente celda se ejecuta el algoritmo genético desarrollado con una entrada de ejemplo. Esta entrada solo contiene 16 de los 20 equipos y es la misma que la que se usa en el algoritmo por fuerza bruta, para poder comparar los resultados.

In [ ]:
partidos_ejemplo = [
    ('Celta', 'Real Madrid'),
    ('Valencia', 'R. Sociedad'),
    ('Mallorca', 'Eibar'),
    ('Athletic', 'Barcelona'),
    ('Leganés', 'Osasuna'),
    ('Villarreal', 'Granada'),
    # ('Alavés', 'Levante'),
    # ('Espanyol', 'Sevilla'),
    ('Betis', 'Valladolid'),
    ('Atlético', 'Getafe')
]

planificador_genetico(partidos_ejemplo)

Planificación mediante ALGORITMO GENÉTICO
-----
Horario: V20, Equipos: Mallorca - Eibar
Horario: S16, Equipos: Villarreal - Granada
Horario: D20, Equipos: Celta - Real Madrid
Horario: S18, Equipos: Atlético - Getafe
Horario: S20, Equipos: Valencia - R. Sociedad
Horario: D16, Equipos: Betis - Valladolid
Horario: L20, Equipos: Leganés - Osasuna
Horario: D18, Equipos: Athletic - Barcelona
-----
Mejor audiencia:  5.8885000000000005 millones de espectadores


En la siguiente celda se ejecuta con los 10 partidos de la documentación del problema.

In [ ]:
partidos_ejemplo = [
    ('Celta', 'Real Madrid'),
    ('Valencia', 'R. Sociedad'),
    ('Mallorca', 'Eibar'),
    ('Athletic', 'Barcelona'),
    ('Leganés', 'Osasuna'),
    ('Villarreal', 'Granada'),
    ('Alavés', 'Levante'),
    ('Espanyol', 'Sevilla'),
    ('Betis', 'Valladolid'),
    ('Atlético', 'Getafe')
]

planificador_genetico(partidos_ejemplo)

Planificación mediante ALGORITMO GENÉTICO
-----
Horario: S16, Equipos: Alavés - Levante
Horario: L20, Equipos: Mallorca - Eibar
Horario: S12, Equipos: Villarreal - Granada
Horario: D18, Equipos: Celta - Real Madrid
Horario: S20, Equipos: Valencia - R. Sociedad
Horario: V20, Equipos: Leganés - Osasuna
Horario: D16, Equipos: Espanyol - Sevilla
Horario: S18, Equipos: Atlético - Getafe
Horario: D20, Equipos: Athletic - Barcelona
Horario: D12, Equipos: Betis - Valladolid
-----
Mejor audiencia:  6.856000000000001 millones de espectadores


## Comparación de los dos algoritmos
Cabe destacar que los dos algoritmos desarrollados tienen funcionamientos parecidos en general, aunque el voraz siempre será determinista mientras que el genético tiene un componente más aleatorio.

Esto se puede comprobar con la **gestión de las coincidencias horarias**: mientras que el voraz suele evitar las coincidencias si es posible, el genético las emplea y gracias a ello puede encontrar resultados con una audiencia ligeramente mayor en algunos casos. Por ejemplo, para la entrada mostrada a continuación, tenemos los resultados que se muestran.

In [ ]:
entrada_comparacion = [
    ('Sevilla', 'Valladolid'),
    ('Celta', 'Valencia'),
    ('Leganés', 'Mallorca'),
    ('Barcelona', 'Eibar'),
    ('Betis', 'Real Madrid'),
    ('Osasuna', 'R. Sociedad'),
    ('Espanyol', 'Levante'),
    ('Getafe', 'Villarreal'),
    ('Alavés', 'Atlético'),
    ('Athletic', 'Granada')
 ]

In [ ]:
%%time
planificador_voraz(entrada_comparacion)

Planificación mediante ALGORITMO VORAZ
-----
Horario: V20, Equipos: Sevilla - Valladolid
Horario: S12, Equipos: Celta - Valencia
Horario: S16, Equipos: Espanyol - Levante
Horario: S18, Equipos: Getafe - Villarreal
Horario: S20, Equipos: Barcelona - Eibar
Horario: D12, Equipos: Athletic - Granada
Horario: D16, Equipos: Alavés - Atlético
Horario: D18, Equipos: Osasuna - R. Sociedad
Horario: D20, Equipos: Betis - Real Madrid
Horario: L20, Equipos: Leganés - Mallorca
-----
Mejor audiencia:  6.4955 millones de espectadores
CPU times: user 8.07 ms, sys: 0 ns, total: 8.07 ms
Wall time: 10.8 ms


In [ ]:
%%time
planificador_genetico(entrada_comparacion)

Planificación mediante ALGORITMO GENÉTICO
-----
Horario: D16, Equipos: Celta - Valencia
Horario: S20, Equipos: Espanyol - Levante
Horario: S18, Equipos: Barcelona - Eibar
Horario: S20, Equipos: Getafe - Villarreal
Horario: D18, Equipos: Osasuna - R. Sociedad
Horario: S12, Equipos: Sevilla - Valladolid
Horario: V20, Equipos: Athletic - Granada
Horario: S16, Equipos: Alavés - Atlético
Horario: D20, Equipos: Betis - Real Madrid
Horario: L20, Equipos: Leganés - Mallorca
-----
Mejor audiencia:  6.5055 millones de espectadores
CPU times: user 16.4 s, sys: 44.8 ms, total: 16.4 s
Wall time: 16.6 s


La diferencia en el ejemplo mostrado anteriormente es de tan solo 0.01 millones de espectadores, pero es interesante cómo en algunos casos las coincidencias horarias son más beneficiosas que otras opciones. También se han encontrado algunos casos donde el resultado del genético es ligeramente inferior que el del voraz.

Por el otro lado, cabe destacar que el **tiempo de ejecución** del algoritmo voraz es considerablemente menor que el del genético: 10.8 milisegundos para el primero comparado con 16.6 segundos para el segundo.

# (*) Calcula la complejidad del algoritmo.

La complejidad de ambos algoritmos desarrollados para mejorar el algoritmo por fuerza bruta será lineal.

## Complejidad del Algoritmo Voraz
El algoritmo se ejecutará $n$ veces, una por cada pareja de equipos. Dentro de cada iteración, se iterará 10 veces, una por cada uno de los horarios disponibles, y que es un número fijo. Por tanto, la complejidad del algoritmo voraz será lineal: $O(n)$.

## Complejidad del Algoritmo Genético

La complejidad del Algoritmo Genético es ligeramente más difícil de calcular. No se han encontrado respuestas claras en artículos científicos sobre un criterio unificado para calcular la complejidad de los algoritmos genéticos debido a que estos son altamente personalizados. Por ello, la solución a esta pregunta se ha desarrollado teniendo en cuenta algunas preguntas y respuestas en foros sobre investigación y desarrollo ([[1]](https://stackoverflow.com/questions/9146086/time-complexity-of-genetic-algorithm), [[2]](https://www.researchgate.net/post/How_Can_I_compute_the_time_complexity_Big-O_Notation_of_Genetic_Algorithm2)).

Cada vez que se genera un cromosoma y se calcula su función *fitness*, se realizan $n$ ejecuciones (el número de partidos que hay). Esto se ejecuta por cada individuo que se genera (teniendo un número N de individuos) a lo largo de G generaciones, donde tras los cruces y las mutaciones es necesario volver a calcular la función *fitness*. A un alto nivel, se tendrían un total de $n \cdot N$ ejecuciones para la creación de la población inicial de $N$ individuos, y dentro de cada generación G se calcularía la función *fitness* para los descendientes: $G \cdot n$. Sumado, nos quedarían $n \cdot (N + G)$ operaciones, con una complejidad final de $O(n)$.

# Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios.

Se genera una entrada aleatoria dados los equipos de La Liga.

In [ ]:
import random

def entrada_datos_aleatorios():

    EQUIPOS = [
        'Real Madrid', 'R. Sociedad', 'Barcelona', 'Celta', 'Valencia', 'Athletic', 'Villarreal', 'Alavés', 'Levante',
        'Espanyol', 'Sevilla', 'Betis', 'Atlético', 'Getafe', 'Mallorca', 'Eibar', 'Leganés', 'Osasuna', 'Granada', 'Valladolid'
    ]

    # Se crean las tuplas aleatorias
    random.shuffle(EQUIPOS)
    RANDOM_MATCHES = [(EQUIPOS[i], EQUIPOS[i + 1]) for i in range(0, len(EQUIPOS), 2)]
    RANDOM_MATCHES = [tuple(sorted(match)) for match in RANDOM_MATCHES]

    return RANDOM_MATCHES

entrada_aleatoria = entrada_datos_aleatorios()
entrada_aleatoria

[('Atlético', 'Betis'),
 ('Getafe', 'Mallorca'),
 ('Sevilla', 'Valencia'),
 ('Valladolid', 'Villarreal'),
 ('Alavés', 'Real Madrid'),
 ('Granada', 'R. Sociedad'),
 ('Celta', 'Osasuna'),
 ('Barcelona', 'Espanyol'),
 ('Athletic', 'Eibar'),
 ('Leganés', 'Levante')]

# Aplica el algoritmo al juego de datos aleatorio generado.

Se aplican los juegos de datos aleatorios generados para los algoritmos en celdas de código separadas y, además, se mide el tiempo de ejecución.

En el caso del algoritmo por fuerza bruta, este no llegó a terminar de ejecutarse en ninguno de los intentos, por lo que se paró manualmente.

In [ ]:
%%time
planificador_bruta(entrada_aleatoria)

KeyboardInterrupt: ignored

In [ ]:
%%time
planificador_voraz(entrada_aleatoria)

Planificación mediante ALGORITMO VORAZ
-----
Horario: V20, Equipos: Getafe - Mallorca
Horario: S12, Equipos: Valladolid - Villarreal
Horario: S16, Equipos: Celta - Osasuna
Horario: S18, Equipos: Atlético - Betis
Horario: S20, Equipos: Alavés - Real Madrid
Horario: D12, Equipos: Athletic - Eibar
Horario: D16, Equipos: Sevilla - Valencia
Horario: D18, Equipos: Granada - R. Sociedad
Horario: D20, Equipos: Barcelona - Espanyol
Horario: L20, Equipos: Leganés - Levante
-----
Mejor audiencia:  6.719999999999999 millones de espectadores
CPU times: user 8.46 ms, sys: 2 µs, total: 8.46 ms
Wall time: 11.7 ms


In [ ]:
%%time
planificador_genetico(entrada_aleatoria)

Planificación mediante ALGORITMO GENÉTICO
-----
Horario: D16, Equipos: Atlético - Betis
Horario: V20, Equipos: Getafe - Mallorca
Horario: S12, Equipos: Valladolid - Villarreal
Horario: D18, Equipos: Granada - R. Sociedad
Horario: S20, Equipos: Alavés - Real Madrid
Horario: S18, Equipos: Sevilla - Valencia
Horario: S16, Equipos: Celta - Osasuna
Horario: D12, Equipos: Athletic - Eibar
Horario: D20, Equipos: Barcelona - Espanyol
Horario: L20, Equipos: Leganés - Levante
-----
Mejor audiencia:  6.720000000000001 millones de espectadores
CPU times: user 15.6 s, sys: 34.9 ms, total: 15.7 s
Wall time: 15.8 s


# Enumera las referencias que has utilizado (si ha sido necesario) para llevar a cabo el trabajo.

La base de la práctica han sido las clases de la asignatura de Algoritmos de Optimización, tanto las videoconferencias teóricas como las actividades guiadas. Además, el manual de la asignatura también fue de gran ayuda para algunos conceptos.

Aparte del material mencionado, también se han utilizado páginas web y foros donde se explican nociones de la complejidad, sobre todo. Los recursos utilizados fueron los siguientes:

[1] https://stackoverflow.com/questions/9146086/time-complexity-of-genetic-algorithm

[2] https://www.researchgate.net/post/How_Can_I_compute_the_time_complexity_Big-O_Notation_of_Genetic_Algorithm2

# Describe brevemente en unas líneas cómo crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño.

A lo largo del desarrollo de esta práctica, surgieron varias ideas de cómo mejorar o avanzar con este caso de uso.
- **Planificación de una temporada entera**. Se trataría de una ampliación del problema, para que en vez de planificar solamente una jornada, se generaran las combinaciones para toda la temporada. La tarea más complicada sería la de generar las posibles combinaciones para cada jornada, pero una vez se tenga eso, sería iterar en cada jornada con las posibles parejas de equipo y los algoritmos desarrollados en esta práctica servirían.
- **Categorías variables de los equipos**. Actualmente, se están manteniendo la categoría de cada equipo fijo. Sin embargo, sería interesante una extensión del problema donde se tuvieran en cuenta otras variables como los resultados de cada equipo para que pudiesen cambiar de categoría a lo largo de la temporada.
- **Información adicional de los equipos**. Podría ser interesante saber qué relaciones hay entre los equipos, para detectar qué partidos son derbi y cuáles no. En equipos de las mismas categorías pueden tener más audiencia los que se consideren derbi. Por ejemplo, R. Sociedad, Real Madrid y Barcelona son de la categoría A, pero es probable que genere más audiencia un partido Real Madrid - Barcelona que otro partido entre R. Sociedad - Barcelona.
- **Posibilidad de incluir información sobre eventos relevantes**. Esto sería útil a la hora de tener en cuenta los coeficientes de cada horario. Por ejemplo, si un domingo a las 20h hay una final de una competición importante de baloncesto, podría considerarse penalizar ese horario según un determinado coeficiente, porque sería probable que la audiencia bajara.
- **Uso en producción**. Para un posible uso en un sistema de producción, sería interesante mantener los dos algoritmos desarrollados y utilizar siempre la salida con mejor resultado de las dos. Muchas veces serán la misma, pero la componente aleatoria del algoritmo genético puede proporcionar resultados ligeramente mejores que el voraz. Si se necesita una planificación en tiempos de ejecución muy bajos, se podría emplear el algoritmo voraz directamente también.